In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import spacy

# Load the dataset
df = pd.read_csv("load.csv")
df.head()

,id,bullying_words,type_bully
0,1,Hate,bully
1,2,Insult,bully
2,3,Threat,bully
3,4,Mockery,bully
4,5,Tease,bully


In [5]:
df.tail()

,id,bullying_words,type_bully
463,464,r u gnon-bullynon-bullyd at art,non-bully
464,465,What&;s your worst habit?,non-bully
465,466,hahahaha thanks i think? did you call me bea...,non-bully
466,467,summer :],non-bully
467,468,Bugs Bunny or Mickey Mouse?,non-bully


In [6]:
# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Define the preprocess function to handle NaN values
def preprocess(bullying_words):
    if pd.notnull(bullying_words):  
        doc = nlp(bullying_words)
        filtered_tokens = []
        for token in doc:
            if not token.is_stop and not token.is_punct and not token.like_num:
                filtered_tokens.append(token.lemma_)
        return ' '.join(filtered_tokens)
    else:
        return ''  

# Apply the preprocess function to create a new column 'processed_text'
df_balanced['processed_text'] = df_balanced['bullying_words'].apply(preprocess)


NameError: name 'df_balanced' is not defined

In [7]:
df_balanced.head()

NameError: name 'df_balanced' is not defined

In [24]:
# Under sampling taking random sample
min_sample = 50
df_bully = df[df.type_bully == "bully"].sample(min_sample, random_state=40)
df_non_bully = df[df.type_bully == "non-bully"].sample(min_sample, random_state=40)

# Combine the balanced dataframes
df_balanced = pd.concat([df_bully, df_non_bully], axis=0)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    df_balanced.bullying_words,
    df_balanced.type_bully,
    test_size=0.2,
    random_state=40,
    stratify=df_balanced.type_bully
)

In [25]:
X_train

74     You're nothing but a waste of space. Nobody ne...
364                          i never said i liked pryce.
295                                         pelo e thata
80     You're so pathetic. No wonder nobody respects ...
34     Why do you even bother trying? You'll never be...
                             ...                        
403                          followed! follow me also :)
301                                               flinty
40             You're doing great! Keep up the good work
272                                   kelello e le 'ngoe
324                          I knw!!!!!  I hate that! D:
Name: bullying_words, Length: 80, dtype: object

In [26]:
Y_train

74         bully
364    non-bully
295        bully
80         bully
34         bully
         ...    
403    non-bully
301        bully
40     non-bully
272        bully
324        bully
Name: type_bully, Length: 80, dtype: object

In [27]:
# Create a pipeline with TfidfVectorizer and LogisticRegression
lejone_tfidf = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 3))),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Fit the pipeline on the training data
lejone_tfidf.fit(X_train, Y_train)

# Make predictions on the testing data
Y_pred_tfidf = lejone_tfidf.predict(X_test)

# Evaluate the TF-IDF based model
print(classification_report(Y_test, Y_pred_tfidf))


              precision    recall  f1-score   support

       bully       0.83      1.00      0.91        10
   non-bully       1.00      0.80      0.89        10

    accuracy                           0.90        20
   macro avg       0.92      0.90      0.90        20
weighted avg       0.92      0.90      0.90        20



In [28]:
# Make predictions on the testing data
Y_pred = lejone_tfidf.predict(X_test)
# Evaluate the model producing a classification report
print(classification_report(Y_test, Y_pred))

from sklearn import metrics
metrics.accuracy_score(Y_test, Y_pred)*100

              precision    recall  f1-score   support

       bully       0.83      1.00      0.91        10
   non-bully       1.00      0.80      0.89        10

    accuracy                           0.90        20
   macro avg       0.92      0.90      0.90        20
weighted avg       0.92      0.90      0.90        20



90.0

In [29]:
from joblib import dump, load
# Save the model to a file
dump(lejone_tfidf, 'kha90.joblib')

['kha90.joblib']